<a href="https://colab.research.google.com/github/MLearners21/NTU_ML_Final_Project2021/blob/main/MLfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Import the important Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC  
from scipy.stats import uniform, truncnorm, randint
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [9]:
# Load both the train and test
train = pd.read_csv('data_train.csv')
test = pd.read_csv('data_test.csv')

# concatenate the two sets and add and indicator column
#data = pd.concat([ train, test])
data = pd.concat([train.assign(ind="train"), test.assign(ind="test")])
# ind column will help to separate the test and train after cleaining

print(data.shape)
data.head()

(5635, 36)


,gender,age,married,num_depen,lat,long,pop,score,num_ref,mult_line,offer,phon_serv,int_serv,int_type,onl_sec,onl_back,dev_prot,premium,str_tv,str_movie,str_music,contract,paperless,payment,tenure_month,month_charge,unlimite,avg_gb,avg_long_dist,t_charge,t_refund,t_extra,t_long_dist,t_revenue,churn,ind
0,Male,78,No,0,34.023810,-118.156582,68701,3,0,No,None,No,Yes,DSL,No,No,Yes,No,No,Yes,No,Month-to-Month,Yes,Bank Withdrawal,1,39.65,No,8,0.00,39.65,0.00,20.0,0.00,59.65,1.0,train
1,Female,74,Yes,1,34.108833,-118.229715,47534,3,1,Yes,Offer E,Yes,Yes,Fiber Optic,No,Yes,No,No,No,No,No,Month-to-Month,Yes,Bank Withdrawal,8,80.65,Yes,17,48.85,633.30,0.00,0.0,390.80,1024.10,1.0,train
2,Male,71,No,3,34.108833,-118.229715,47534,2,0,Yes,None,Yes,Yes,Fiber Optic,No,No,No,No,No,No,No,Month-to-Month,No,Credit Card,10,98.50,Yes,12,19.76,2514.50,13.43,0.0,494.00,2995.07,1.0,train
3,Female,78,Yes,1,33.936291,-118.332639,27778,2,1,No,Offer C,Yes,Yes,Fiber Optic,No,Yes,Yes,No,Yes,Yes,No,Month-to-Month,Yes,Bank Withdrawal,25,98.50,Yes,12,19.76,2514.50,13.43,0.0,494.00,2995.07,2.0,train
4,Male,80,Yes,1,33.972119,-118.020188,26265,2,1,Yes,Offer C,Yes,Yes,Fiber Optic,No,No,No,No,Yes,No,No,Month-to-Month,Yes,Bank Withdrawal,37,76.50,Yes,18,6.33,2868.15,0.00,0.0,234.21,3102.36,4.0,train


In [10]:
train.isnull().values.any()

False

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5635 entries, 0 to 1408
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         5635 non-null   object 
 1   age            5635 non-null   int64  
 2   married        5635 non-null   object 
 3   num_depen      5635 non-null   int64  
 4   lat            5635 non-null   float64
 5   long           5635 non-null   float64
 6   pop            5635 non-null   int64  
 7   score          5635 non-null   int64  
 8   num_ref        5635 non-null   int64  
 9   mult_line      5635 non-null   object 
 10  offer          5635 non-null   object 
 11  phon_serv      5635 non-null   object 
 12  int_serv       5635 non-null   object 
 13  int_type       5635 non-null   object 
 14  onl_sec        5635 non-null   object 
 15  onl_back       5635 non-null   object 
 16  dev_prot       5635 non-null   object 
 17  premium        5635 non-null   object 
 18  str_tv  

In [12]:
data.dtypes

gender            object
age                int64
married           object
num_depen          int64
lat              float64
long             float64
pop                int64
score              int64
num_ref            int64
mult_line         object
offer             object
phon_serv         object
int_serv          object
int_type          object
onl_sec           object
onl_back          object
dev_prot          object
premium           object
str_tv            object
str_movie         object
str_music         object
contract          object
paperless         object
payment           object
tenure_month       int64
month_charge     float64
unlimite          object
avg_gb             int64
avg_long_dist    float64
t_charge         float64
t_refund         float64
t_extra          float64
t_long_dist      float64
t_revenue        float64
churn            float64
ind               object
dtype: object

In [13]:
data.describe()

,age,num_depen,lat,long,pop,score,num_ref,tenure_month,month_charge,avg_gb,avg_long_dist,t_charge,t_refund,t_extra,t_long_dist,t_revenue,churn
count,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,4226.000000
mean,46.806921,0.398048,36.224968,-119.775177,22329.400887,3.250932,1.536469,32.160248,64.869443,20.170364,24.248130,2270.353469,2.177627,6.769867,787.539716,3022.787764,0.597018
std,15.317200,0.886151,2.487805,2.157003,21054.705649,1.197944,2.695243,24.534431,31.221126,20.320202,21.372571,2255.008913,8.956667,24.406575,1083.985846,2849.528739,1.229299
min,19.000000,0.000000,32.555828,-124.301372,11.000000,1.000000,0.000000,0.000000,6.965217,0.000000,0.000000,18.850000,0.000000,0.000000,0.000000,21.400000,0.000000
25%,36.000000,0.000000,33.994524,-121.825544,2633.000000,3.000000,0.000000,8.000000,35.400000,3.000000,9.215000,393.625000,0.000000,0.000000,72.460000,594.290000,0.000000
50%,47.000000,0.000000,36.292230,-119.657092,17929.000000,3.000000,0.000000,29.000000,70.450000,16.000000,23.040000,1397.300000,0.000000,0.000000,410.400000,2112.100000,0.000000
75%,57.000000,0.000000,38.197202,-117.992066,36325.000000,4.000000,2.000000,55.000000,89.800000,27.000000,36.480000,3765.625000,0.000000,0.000000,1197.210000,4746.970000,1.000000
max,80.000000,9.000000,41.962127,-114.192901,105285.000000,5.000000,10.000000,75.000000,422.758333,85.000000,273.532222,8594.400000,149.000000,150.000000,19694.320000,11979.340000,5.000000


In [14]:
for item in data.columns:
    print(item)
    print (data[item].unique())

gender
['Male' 'Female']
age
[78 74 71 80 72 76 65 77 70 68 75 47 69 73 67 79 66 37 24 23 38 29 41 34
 21 64 53 63 32 26 48 39 22 40 30 54 59 49 25 51 33 31 44 50 28 20 57 19
 45 58 27 55 46 61 62 36 43 35 42 60 52 56]
married
['No' 'Yes']
num_depen
[0 1 3 2 5 4 6 9]
lat
[34.02381  34.108833 33.936291 ... 33.801869 33.313828 34.758699]
long
[-118.156582 -118.229715 -118.332639 ... -117.995068 -116.940501
 -120.275839]
pop
[68701 47534 27778 ... 29694  2615  1328]
score
[3 2 1 5 4]
num_ref
[ 0  1  2  3  7  4  6  5 10  9  8]
mult_line
['No' 'Yes']
offer
['None' 'Offer E' 'Offer C' 'Offer D' 'Offer B' 'Offer A']
phon_serv
['No' 'Yes']
int_serv
['Yes' 'No']
int_type
['DSL' 'Fiber Optic' 'None' 'Cable']
onl_sec
['No' 'Yes']
onl_back
['No' 'Yes']
dev_prot
['Yes' 'No']
premium
['No' 'Yes']
str_tv
['No' 'Yes']
str_movie
['Yes' 'No']
str_music
['No' 'Yes']
contract
['Month-to-Month' 'Two Year' 'One Year']
paperless
['Yes' 'No']
payment
['Bank Withdrawal' 'Credit Card' 'Mailed Check']
tenure_mon

In [15]:
#Convert all yes and no to 0's & 1's so our classifier can use this data.

data["gender"].replace(['Female','Male'],[0,1],inplace=True)
data["married"].replace(['No', 'Yes'], [0, 1], inplace=True)


data["phon_serv"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["paperless"].replace(['No', 'Yes'], [0, 1], inplace=True)

data["str_movie"].replace(['No', 'Yes'], [0, 1], inplace=True)

data["int_type"].replace(['Fiber Optic', 'Cable', 'None', 'DSL'],[0,1,2, 3],inplace=True)

data["contract"].replace(['One Year', 'Month-to-Month', 'Two Year'],[0,1,2],inplace=True)

#data = pd.get_dummies(data=data, columns=['Payment Method'])


data["payment"].replace(['Bank Withdrawal', 'Credit Card', 'Mailed Check'],[ 0, 1, 2], inplace=True)
data["mult_line"].replace(['No','Yes'],[0,1],inplace=True)
data["onl_sec"].replace(['No','Yes'],[0,1],inplace=True)
data["onl_back"].replace(['No','Yes'],[0,1],inplace=True)
data["dev_prot"].replace(['No','Yes'],[0,1],inplace=True)
data["premium"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["str_tv"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["str_music"].replace(['No', 'Yes'], [0, 1], inplace=True)

data["unlimite"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["int_serv"].replace(['No', 'Yes'], [0, 1], inplace=True)
#data["Referred a Friend"].replace(['No', 'Yes'], [0, 1], inplace=True)


data["offer"].replace(['None', 'Offer E', 'Offer C', 'Offer A' ,'Offer B', 'Offer D'],[ 0, 1, 2, 3, 4, 5], inplace=True)

data.head()


,gender,age,married,num_depen,lat,long,pop,score,num_ref,mult_line,offer,phon_serv,int_serv,int_type,onl_sec,onl_back,dev_prot,premium,str_tv,str_movie,str_music,contract,paperless,payment,tenure_month,month_charge,unlimite,avg_gb,avg_long_dist,t_charge,t_refund,t_extra,t_long_dist,t_revenue,churn,ind
0,1,78,0,0,34.023810,-118.156582,68701,3,0,0,0,0,1,3,0,0,1,0,0,1,0,1,1,0,1,39.65,0,8,0.00,39.65,0.00,20.0,0.00,59.65,1.0,train
1,0,74,1,1,34.108833,-118.229715,47534,3,1,1,1,1,1,0,0,1,0,0,0,0,0,1,1,0,8,80.65,1,17,48.85,633.30,0.00,0.0,390.80,1024.10,1.0,train
2,1,71,0,3,34.108833,-118.229715,47534,2,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,1,10,98.50,1,12,19.76,2514.50,13.43,0.0,494.00,2995.07,1.0,train
3,0,78,1,1,33.936291,-118.332639,27778,2,1,0,2,1,1,0,0,1,1,0,1,1,0,1,1,0,25,98.50,1,12,19.76,2514.50,13.43,0.0,494.00,2995.07,2.0,train
4,1,80,1,1,33.972119,-118.020188,26265,2,1,1,2,1,1,0,0,0,0,0,1,0,0,1,1,0,37,76.50,1,18,6.33,2868.15,0.00,0.0,234.21,3102.36,4.0,train


In [16]:
# Our data is clean and now we can split back to test and train before Training the model

train_set, test_set = data[data["ind"].eq("train")], data[data["ind"].eq("test")]

X_train = train_set.loc[:,'gender': 't_revenue']
y_train = train_set["churn"]


X_test = test_set.loc[:,'gender': 't_revenue']
# y_test = test_set["churn"]

print(X_test.shape)
print(y_train.shape)

(1409, 34)
(4226,)


In [17]:
allData = pd.read_csv("data_all.csv")

customerId = allData.loc[allData['is_test'] == True]
testId = customerId['id'] 
#print(testId)

In [18]:
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.02, random_state = 42)'''

'\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.02, random_state = 42)'

In [12]:
#n_estimators = [10,30, 40, 80, 50,100, 150] # number of trees in the random forest
n_estimators =randint(4,200)
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [5, 10, 15, 20, 30] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4, 5, 6, 10] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [14]:
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
               n_iter = 10, cv = 100, verbose=2, random_state=35, n_jobs = -1)

In [15]:
rf_random.fit(X_train, y_train)

Fitting 100 folds for each of 10 candidates, totalling 1000 fits


RandomizedSearchCV(cv=100, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4, 5, 6, 10],
                                        'min_samples_split': [5, 10, 15, 20,
                                                              30],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f152b2f16d0>},
                   random_state=35, verbose=2)

In [16]:
print ('Random grid: ', random_grid, '\n')
# print the best parameters
print ('Best Parameters: ', rf_random.best_params_, ' \n')

Random grid:  {'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f152b2f16d0>, 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120], 'min_samples_split': [5, 10, 15, 20, 30], 'min_samples_leaf': [1, 3, 4, 5, 6, 10], 'bootstrap': [True, False]} 

Best Parameters:  {'bootstrap': False, 'max_depth': 110, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 15, 'n_estimators': 166}  



In [17]:
# Use the best parameters
randmf = RandomForestClassifier(n_estimators = 150, min_samples_split = 26, min_samples_leaf= 1, max_features = 'sqrt', max_depth= 80, bootstrap=True) 
randmf.fit( X_train, y_train) 


RandomForestClassifier(max_depth=80, max_features='sqrt', min_samples_split=26,
                       n_estimators=150)

In [18]:
y_pred_rf1 = pd.DataFrame( {
"predicted": randmf.predict( 
( X_test ) ) } ) 
y_pred_rf1

# np.savetxt('RFtry1.csv', y_pred_rf1,header="TestIds, actual, predicted", delimiter=",",fmt='%i' )

submission = pd.DataFrame({"Customer ID": testId, "Churn Category":randmf.predict(X_test)})
submission.to_csv('submissionRF.csv', index=False)

In [19]:
'''
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred_rf1)

#accuracy=accuracy_score(test_survived['Survived'],predictions)
print(accuracy)'''

"\nfrom sklearn.metrics import accuracy_score\naccuracy = accuracy_score(y_test,y_pred_rf1)\n\n#accuracy=accuracy_score(test_survived['Survived'],predictions)\nprint(accuracy)"

In [20]:
# View confusion matrix for test data and predictions
#confusion_matrix(y_test, y_pred_rf1)

In [21]:
# View the classification report for test data and predictions
#print(classification_report(y_test, y_pred_rf1))

In [22]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto")
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto")
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto")
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto")

In [23]:
for i in range(4):
    # Separate data into test and training sets
    X_train, X_test, y_train #, y_test = train_test_split(X, y, test_size = 0.20)# Train a SVC model using different kernal
    svclassifier = getClassifier(i) 
    svclassifier.fit(X_train, y_train)# Make prediction
    y_pred = svclassifier.predict(X_test)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    #print(classification_report(y_test,y_pred))

Evaluation: Polynomial kernel
Evaluation: RBF kernel
Evaluation: Sigmoid kernel
Evaluation: Linear kernel


In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC  

In [23]:
param_grid = {'C': [0.001, 0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001,0.0001],'kernel': ['rbf', 'poly', 'sigmoid']}

In [ ]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV] END .......................C=0.001, gamma=1, kernel=rbf; total time=   0.5s
[CV] END .......................C=0.001, gamma=1, kernel=rbf; total time=   0.5s
[CV] END .......................C=0.001, gamma=1, kernel=rbf; total time=   0.5s
[CV] END .......................C=0.001, gamma=1, kernel=rbf; total time=   0.5s
[CV] END .......................C=0.001, gamma=1, kernel=rbf; total time=   0.5s


In [ ]:
print(grid.best_estimator_)

In [ ]:
grid_predictions = grid.predict(X_test)

submission = pd.DataFrame({"Customer ID": testId, "Churn Category":randmf.predict(X_test)})
submission.to_csv('submissionSVM.CSV', index=False)
print(grid_predictions)
#print(classification_report(y_test,grid_predictions))